In [3]:
# IMPORTAR LIBRERÍAS

import cv2
from cv2 import dnn_superres
from tqdm.notebook import tqdm
import os

In [ ]:
# Función de escalado
def upscale(sr, filename, source_folder, output_folder):
    output = f"{output_folder}/{filename}"
    if os.path.exists(output):
        return
    image = cv2.imread(f"{source_folder}/{filename}")
    result = sr.upsample(image)
    cv2.imwrite(output, result)

In [ ]:
# Establecer modelo
sr = dnn_superres.DnnSuperResImpl_create()
model_path = "models/ESPCN_x3.pb"
sr.readModel(model_path)
sr.setModel("espcn", 3)

In [ ]:
input_folder = "./Letras"
output_folder = "./Letras3K"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

dirlist = next(os.walk(f"{input_folder}"))[1]
for dirname in tqdm(list(dirlist)):
    source = f"{input_folder}/{dirname}"
    target = f"{output_folder}/{dirname}"
    if not os.path.exists(target):
        os.makedirs(target)
    included_extensions = (".jpg", ".jpeg", ".bmp", ".png")
    imgfiles = [fn for fn in os.listdir(source) if fn.endswith(included_extensions)]
    
    for imgfile in imgfiles:
        upscale(sr,imgfile,source,target)


In [5]:
# Mejorar calidad
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool

def improve(filename, source_folder, output_folder):
    output = f"{output_folder}/{filename}"
    if os.path.exists(output):
        return
    image = cv2.imread(f"{source_folder}/{filename}")
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    image = cv2.filter2D(image, -2, kernel) #nitidez
    image = cv2.fastNlMeansDenoisingColored(image, None, 20, 20) #ruido
    cv2.imwrite(output, image)

input_folder = "./raw"
output_folder = "./BetterLetras2K/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

dirlist = next(os.walk(f"{input_folder}"))[1]
for dirname in tqdm(list(dirlist)):
    
    source = f"{input_folder}/{dirname}"
    target = f"{output_folder}/{dirname}"
    if not os.path.exists(target):
        os.makedirs(target)
    included_extensions = (".jpg", ".jpeg", ".bmp", ".png")
    imgfiles = [fn for fn in os.listdir(source) if fn.endswith(included_extensions)]
    imgpaths = []
    pool = ThreadPool(processes=12)
    for imgfile in imgfiles:
        pool.apply_async(improve, args=(imgfile,source,target))
    pool.close()
    pool.join()

  0%|          | 0/100 [00:00<?, ?it/s]